In [1]:
! pip install qiskit
! pip install qiskit-aer
! pip install pylatexenc


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import Aer  # Corrected import for Aer simulator
from qiskit.circuit.library import QFT
import numpy as np

# Use Aer simulator backend
backend = Aer.get_backend('qasm_simulator')

     


# PART 1
# Create the quantum circuit with 3 qubits and 3 classical bits
q = QuantumRegister(3, 'q')  # Quantum register
c0 = ClassicalRegister(1, 'c0')  # Classical register for Alice's qubit
c1 = ClassicalRegister(1, 'c1')  # Classical register for Bob's qubit
c2 = ClassicalRegister(1, 'c2')  # Classical register for the result
circuit = QuantumCircuit(q, c0, c1, c2)

# Prepare the initial state to be teleported
circuit.initialize([0, 1], q[0])  # Initialize q[0] in the |1> state
circuit.barrier()

# Create an entanglement between Alice's and Bob's qubits
circuit.h(q[1])  # Hadamard on Bob's qubit
circuit.cx(q[1], q[2])  # CNOT between Bob's qubits
circuit.barrier()

# Teleportation process
circuit.cx(q[0], q[1])  # CNOT for teleportation
circuit.h(q[0])  # Hadamard on Alice's qubit
circuit.barrier()

# Measure Alice's qubits and send the measurement results to Bob
circuit.measure(q[0], c0[0])  # Measure Alice's qubit
circuit.measure(q[1], c1[0])  # Measure Bob's qubit

# Apply corrective operations on Bob's qubit based on the measurement results
circuit.x(q[2]).c_if(c1, 1)  # Apply X gate if Bob's measurement is 1
circuit.z(q[2]).c_if(c0, 1)  # Apply Z gate if Alice's measurement is 1

# Measure the teleported qubit
circuit.measure(q[2], c2[0])  # Measure the teleported state

# Visualize the circuit
print(circuit)
circuit.draw(output='mpl')

# Simulate the circuit using the QASM simulator
transpiled_circuit = transpile(circuit, backend)  # Transpile the circuit for the simulator
job = backend.run(transpiled_circuit, shots=1)  # Run the transpiled circuit
result = job.result()
teleported_state = result.get_counts(transpiled_circuit)  # Get counts from the transpiled circuit

# Print the teleported state
print("Teleported state:", teleported_state)

      ┌─────────────────┐ ░            ░      ┌───┐ ░ ┌─┐                    
 q_0: ┤ Initialize(0,1) ├─░────────────░───■──┤ H ├─░─┤M├────────────────────
      └─────────────────┘ ░ ┌───┐      ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐                 
 q_1: ────────────────────░─┤ H ├──■───░─┤ X ├──────░──╫─┤M├─────────────────
                          ░ └───┘┌─┴─┐ ░ └───┘      ░  ║ └╥┘ ┌───┐  ┌───┐ ┌─┐
 q_2: ────────────────────░──────┤ X ├─░────────────░──╫──╫──┤ X ├──┤ Z ├─┤M├
                          ░      └───┘ ░            ░  ║  ║  └─╥─┘  └─╥─┘ └╥┘
                                                       ║  ║    ║   ┌──╨──┐ ║ 
c0: 1/═════════════════════════════════════════════════╩══╬════╬═══╡ 0x1 ╞═╬═
                                                       0  ║ ┌──╨──┐└─────┘ ║ 
c1: 1/════════════════════════════════════════════════════╩═╡ 0x1 ╞════════╬═
                                                          0 └─────┘        ║ 
c2: 1/══════════════════════════════════════════════════════════

In [3]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import Aer  # Corrected import for Aer simulator

from qiskit.circuit.library import QFT

import numpy as np



# Initialize quantum and classical registers
q = QuantumRegister(5, 'q')
c = ClassicalRegister(5, 'c')

# Create the quantum circuit
circuit = QuantumCircuit(q, c)

# Prepare the circuit
circuit.x(q[4])  # Apply X gate to q[4]
circuit.x(q[2])  # Apply X gate to q[2]
circuit.x(q[0])  # Apply X gate to q[0]
circuit.append(QFT(num_qubits=5, approximation_degree=0, do_swaps=True, inverse=False, insert_barriers=False), q)
circuit.measure(q, c)  # Measure the qubits
circuit.draw(output='mpl', filename='qft1.png')  # Draw the circuit and save to a file
print(circuit)

# Transpile the circuit for the backend
transpiled_circuit = transpile(circuit, backend)

# Run the job
job = backend.run(transpiled_circuit, shots=1000)

# Get the results
result = job.result()
counts = result.get_counts()

# Display the QFT output
print("\nQFT Output")
print("-------------")

print(counts)

     ┌───┐┌──────┐┌─┐            
q_0: ┤ X ├┤0     ├┤M├────────────
     └───┘│      │└╥┘┌─┐         
q_1: ─────┤1     ├─╫─┤M├─────────
     ┌───┐│      │ ║ └╥┘┌─┐      
q_2: ┤ X ├┤2 QFT ├─╫──╫─┤M├──────
     └───┘│      │ ║  ║ └╥┘┌─┐   
q_3: ─────┤3     ├─╫──╫──╫─┤M├───
     ┌───┐│      │ ║  ║  ║ └╥┘┌─┐
q_4: ┤ X ├┤4     ├─╫──╫──╫──╫─┤M├
     └───┘└──────┘ ║  ║  ║  ║ └╥┘
c: 5/══════════════╩══╩══╩══╩══╩═
                   0  1  2  3  4 

QFT Output
-------------
{'11000': 28, '01010': 34, '00001': 36, '10011': 26, '00000': 32, '10001': 30, '01101': 31, '10110': 34, '11001': 31, '10100': 33, '11111': 34, '00111': 26, '01111': 34, '00010': 33, '10000': 42, '11010': 33, '00110': 36, '10111': 32, '01000': 28, '01001': 31, '11101': 23, '01100': 32, '11011': 26, '00101': 32, '01110': 36, '10101': 24, '11100': 30, '01011': 29, '10010': 30, '00100': 32, '11110': 38, '00011': 24}


In [4]:
# PART 3
q = QuantumRegister(5, 'q')
c = ClassicalRegister(5, 'c')

circuit = QuantumCircuit(q, c)

circuit.x(q[4])
circuit.x(q[2])
circuit.x(q[0])
circuit.append(QFT(num_qubits=5, approximation_degree=0, do_swaps=True, inverse=False, insert_barriers=False, name='qft'), q)
circuit.measure(q, c)
circuit.draw(output='mpl', filename='qft1.png')
print(circuit)

# Transpile the circuit for the backend
transpiled_circuit = transpile(circuit, backend)

# Run the job
job = backend.run(transpiled_circuit, shots=1000)

# Get the results
result = job.result()
counts = result.get_counts()

# Display the QFT output
print("\nQFT Output")
print("-------------")
print(counts)


# Create another circuit for QFT with Inverse
q = QuantumRegister(5, 'q')
c = ClassicalRegister(5, 'c')

circuit = QuantumCircuit(q, c)

circuit.x(q[4])
circuit.x(q[2])
circuit.x(q[0])
circuit.append(QFT(num_qubits=5, approximation_degree=0, do_swaps=True, inverse=True, insert_barriers=False, name='qft_inv'), q)
circuit.measure(q, c)
circuit.draw(output='mpl', filename='qft2.png')

print(circuit)

# Transpile the circuit for the backend
transpiled_circuit = transpile(circuit, backend)

# Run the job
job = backend.run(transpiled_circuit, shots=1000)

# Get the results
result = job.result()
counts = result.get_counts()

# Display the QFT output
print("\n QFT with inverse QFT Output")
print("-------------")
print(counts)

     ┌───┐┌──────┐┌─┐            
q_0: ┤ X ├┤0     ├┤M├────────────
     └───┘│      │└╥┘┌─┐         
q_1: ─────┤1     ├─╫─┤M├─────────
     ┌───┐│      │ ║ └╥┘┌─┐      
q_2: ┤ X ├┤2 qft ├─╫──╫─┤M├──────
     └───┘│      │ ║  ║ └╥┘┌─┐   
q_3: ─────┤3     ├─╫──╫──╫─┤M├───
     ┌───┐│      │ ║  ║  ║ └╥┘┌─┐
q_4: ┤ X ├┤4     ├─╫──╫──╫──╫─┤M├
     └───┘└──────┘ ║  ║  ║  ║ └╥┘
c: 5/══════════════╩══╩══╩══╩══╩═
                   0  1  2  3  4 

QFT Output
-------------
{'11101': 32, '10000': 36, '11111': 38, '00111': 28, '00100': 35, '01100': 29, '00110': 36, '00000': 32, '10001': 33, '01110': 27, '00101': 25, '10010': 29, '10111': 35, '01000': 31, '01111': 34, '01011': 33, '00001': 19, '11001': 34, '00011': 26, '11011': 33, '00010': 34, '10110': 29, '01010': 29, '10100': 41, '11000': 26, '11010': 39, '01101': 39, '10101': 35, '11110': 26, '11100': 29, '10011': 29, '01001': 19}
     ┌───┐┌──────────┐┌─┐            
q_0: ┤ X ├┤0         ├┤M├────────────
     └───┘│          │└╥┘┌─┐         
q